In [37]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import numpy as np
import seaborn as sns
import tensorflow as tf
import pathlib
from tensorflow import keras
from keras.layers import Dense,Reshape,Flatten,Input,Conv2D,Concatenate,Lambda,BatchNormalization,ReLU,Conv1D,DepthwiseConv1D
from keras.callbacks import TensorBoard , ModelCheckpoint

mpl.rcParams['figure.figsize'] = (12, 8)
mpl.rcParams['axes.grid'] = False

In [22]:
df = pd.read_csv('data/sp500_with_indicators.csv')
df.pop('Unnamed: 0')
date_time = pd.to_datetime(df.pop('Date'))
timestamp_s = date_time.map(pd.Timestamp.timestamp)

#Removing unnecessary columns

CLOSE = df.pop('Close')
OPEN = df.pop('Open') 
HIGH = df.pop('High') 
LOW = df.pop('Low') 
VOLUME = df.pop('Volume') 
SPREAD = df.pop('Spread')
TICKVOL = df.pop('TickVol') 
BUY_OR_SELL = df.pop('Class')
BUY_OR_SELL_NUMBER = df.pop('Class_Number')
CANDEL_BODY = df.pop('Candel_Body')    
CANDEL_UPPER_SHADOW = df.pop('Candel_Upper_Shadow')   
CANDEL_LOWER_SHADOOW = df.pop('Candel_Lower_Shadow')   

df['CANDEL_UPPER_SHADOW'] = CANDEL_UPPER_SHADOW
df['CANDEL_BODY'] = CANDEL_BODY
df['CANDEL_LOWER_SHADOOW'] = CANDEL_LOWER_SHADOOW

In [23]:
df

,RSI,CCI,STOCH,STOCH_SIGNAL,Boll_Percent,WILL_R,DON_CHIAN_pband,ADX_Pos,ADX_Neg,Aroon,CANDEL_UPPER_SHADOW,CANDEL_BODY,CANDEL_LOWER_SHADOOW
0,53.358846,16.428880,57.272727,37.922351,0.656014,-42.727273,0.614754,20.198086,20.061216,-28.0,0.0,2.5,0.5
1,51.659869,27.991899,48.181818,47.972028,0.579344,-51.818182,0.532787,19.483670,19.351641,-28.0,0.0,-1.0,0.8
2,49.132736,-13.260635,34.545455,46.666667,0.445320,-65.454545,0.409836,18.467570,21.551741,-28.0,0.2,-1.5,0.9
3,45.460502,-77.831342,15.789474,32.838915,0.223298,-84.210526,0.221311,17.521366,23.316732,-24.0,0.2,-2.3,0.0
4,51.862081,-9.465111,56.989247,35.774725,0.627301,-43.010753,0.532787,18.911944,21.167662,-20.0,0.2,3.8,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91516,39.595103,-97.218277,20.886076,15.089552,0.176959,-79.113924,0.208861,14.750090,21.255647,-36.0,2.0,0.0,0.3
91517,43.307788,-78.384883,28.481013,23.417722,0.279226,-71.518987,0.284810,13.800860,19.887757,-36.0,1.0,1.2,0.5
91518,41.043835,-80.791539,22.151899,23.839662,0.237582,-77.848101,0.221519,13.126965,18.916639,-36.0,0.3,-1.0,0.7
91519,39.487705,-82.970358,18.300654,22.977855,0.220795,-81.699346,0.177215,12.471156,19.220558,-36.0,0.8,-0.7,0.5


In [24]:
def normalize(data:pd.core.frame.DataFrame,data_std=None,data_mean=None):
    if(data_std is None):
        data_std = data.std()
    if(data_mean is None):
        data_mean = data.mean()


    data_normalized = (data-data_mean)/data_std

    return data_normalized, data_std,data_mean
    
def min_max_scaler(data:pd.core.frame.DataFrame,minimum:float,maximum:float):
    data_max = data.describe().transpose()['max']
    data_min = data.describe().transpose()['min']
    data_std = (data-data_min)/(data_max-data_min)
    data_scaled = data_std*(abs(minimum)+maximum)+minimum
    return data_scaled


In [5]:
# data_normalized , data_std , data_mean = normalize(df) 

# data_scaled = min_max_scaler(data_normalized,-1,1)

In [6]:
# data_scaled.describe().transpose()



,count,mean,std,min,25%,50%,75%,max
RSI,91521.0,0.042346,0.276058,-1.0,-0.142144,0.045682,0.231140,1.0
CCI,91521.0,0.011600,0.183201,-1.0,-0.126138,0.024106,0.150153,1.0
STOCH,91521.0,0.072877,0.599633,-1.0,-0.454545,0.111111,0.617391,1.0
STOCH_SIGNAL,91521.0,0.072876,0.555830,-1.0,-0.425926,0.103566,0.586667,1.0
Boll_Percent,91521.0,0.012309,0.314085,-1.0,-0.234456,0.032681,0.262322,1.0
WILL_R,91521.0,0.072877,0.599633,-1.0,-0.454545,0.111111,0.617391,1.0
DON_CHIAN_pband,91521.0,0.083743,0.596811,-1.0,-0.439716,0.126214,0.625806,1.0
ADX_Pos,91521.0,-0.383765,0.235053,-1.0,-0.553026,-0.402243,-0.237762,1.0
ADX_Neg,91521.0,-0.379483,0.239576,-1.0,-0.549745,-0.402972,-0.234512,1.0
Aroon,91521.0,0.049111,0.621997,-1.0,-0.541667,0.208333,0.625000,1.0


In [25]:
class dataProcessing():

    def __init__(self ,data:pd.core.frame.DataFrame , output:pd.core.series.Series , input_width:int, stockname:str,
                    min_max :bool =False,minimum:float=1.0 , maximum:float=1.0):

        self.stockname :str  = stockname 
        self.input_width : int = input_width
        self.data :pd.core.frame.DataFrame = data
        self.output : pd.core.series.Series = output
        self.column_indices : list[str] = {name: i for i, name in enumerate(data.columns)}
        self.num_features : int = data.shape[1]
        
        #slit into test and train data
        n = len(data)

        self.input_train_dataset = data[:int(0.90 * n)]
        self.output_train_dataset = output[:int(0.90 * n)]


        self.input_test_dataset = data[int(0.90 * n):]
        self.output_test_dataset = output[int(0.90 * n):]


        #reset indecies
        self.input_test_dataset = self.input_test_dataset.reset_index()
        self.input_test_dataset.pop('index')
        
        self.output_test_dataset = self.output_test_dataset.reset_index()
        self.output_test_dataset = self.output_test_dataset['Class_Number']

        #Normalizing The Data

        self.input_train_dataset, self.input_train_std ,self.input_train_mean = normalize(self.input_train_dataset)
        self.input_test_dataset ,_,_ = normalize(self.input_test_dataset,data_std=self.input_train_std,data_mean=self.input_train_mean)


        # self.input_train_std = self.input_train_dataset.std()  
        # self.input_train_mean = self.input_train_dataset.mean()
        
        # self.input_train_dataset = (self.input_train_dataset - self.input_train_mean) / self.input_train_std
        # self.input_test_dataset = (self.input_test_dataset - self.input_train_mean) / self.input_train_std

        #Min max scaling 

        if(min_max):
            self.input_train_dataset = min_max_scaler(self.input_train_dataset,minimum,maximum)
            self.input_test_dataset = min_max_scaler(self.input_test_dataset,minimum,maximum)
            

    def plot_normalized_data(self):
        data_std = (self.data - self.input_train_mean) / self.input_train_std
        data_std = data_std.melt(var_name='Column', value_name='Normalized')
        plt.figure(figsize=(40, 12))
        ax = sns.violinplot(x='Column', y='Normalized', data=data_std)
        _ = ax.set_xticklabels(self.data.keys(), rotation=90)

    def make_windows(self,input_data:pd.core.frame.DataFrame , output_data:pd.core.series.Series):
        
        window_input = []
        window_output=[]

        for i in range(self.input_width,len(input_data)):

            window_input.append(input_data[i-self.input_width:i].reset_index())
            
            window_output.append(output_data[i])

            window_input[-1].pop('index')
            
            #convert pd.DataFrame to numpy
            window_input[-1]= window_input[-1].to_numpy() 

        #convert list to numpy
        window_input = np.asarray(window_input)
        window_output = np.asarray(window_output)

        window_output = tf.one_hot(window_output,depth=2)

        return window_input,window_output  



In [26]:
data = dataProcessing(  data=df,
                        output=BUY_OR_SELL_NUMBER,
                        input_width=256,
                        stockname='sp500_with_Indicator',
                        min_max=True,
                        minimum=-1.0,
                        maximum=1.0
                        )


In [27]:
input_window , output_window = data.make_windows(   
                                                    input_data=data.input_train_dataset,
                                                    output_data=data.output_train_dataset,
                                                )

In [34]:
def compile_and_fit(model,modelname,data:dataProcessing,input_window,output_window):
    MAX_EPOCHS = 25

    #check if path is available
    path = f'models/{modelname}/{data.stockname}/tensorboard/logs/fit'
    pathlib.Path(path).mkdir(parents=True,exist_ok=True)

    
    log_dir =f'models/{modelname}/{data.stockname}/tensorboard/logs/fit/{datetime.datetime.now().strftime("(%Y-%m-%d)-(%H-%M-%S)")}'
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


    model.compile(loss='binary_crossentropy',
                    optimizer='adam',
                    metrics=[tf.metrics.BinaryAccuracy()])

    history = model.fit(    x=input_window,
                            y= output_window,
                            validation_split=0.1,
                            epochs=MAX_EPOCHS,
                            verbose=1,
                            callbacks=[tensorboard_callback],
                            batch_size= 64
                        )
    
    return history

In [ ]:
def convert_candel_to_1d_array(candels_reshape,filters):

    conv2d_1 = Conv2D(filters=filters,kernel_size=(  4,3),strides=(2,1))(candels_reshape)
    conv2d_2 = Conv2D(filters=filters,kernel_size=(  8,3),strides=(2,1))(candels_reshape)
    conv2d_3 = Conv2D(filters=filters,kernel_size=( 16,3),strides=(2,1))(candels_reshape)
    conv2d_4 = Conv2D(filters=filters,kernel_size=( 32,3),strides=(2,1))(candels_reshape)
    conv2d_5 = Conv2D(filters=filters,kernel_size=( 64,3),strides=(2,1))(candels_reshape)
    conv2d_6 = Conv2D(filters=filters,kernel_size=(128,3),strides=(2,1))(candels_reshape)

    batch_norm_1 = BatchNormalization()(conv2d_1)
    batch_norm_2 = BatchNormalization()(conv2d_2)
    batch_norm_3 = BatchNormalization()(conv2d_3)
    batch_norm_4 = BatchNormalization()(conv2d_4)
    batch_norm_5 = BatchNormalization()(conv2d_5)
    batch_norm_6 = BatchNormalization()(conv2d_6)

    relu_1 = ReLU()(batch_norm_1)
    relu_2 = ReLU()(batch_norm_2)
    relu_3 = ReLU()(batch_norm_3)
    relu_4 = ReLU()(batch_norm_4)
    relu_5 = ReLU()(batch_norm_5)
    relu_6 = ReLU()(batch_norm_6)

    reshape_1 = Reshape(target_shape=(relu_1.shape[1],relu_1.shape[3]))(relu_1)
    reshape_2 = Reshape(target_shape=(relu_2.shape[1],relu_2.shape[3]))(relu_2)
    reshape_3 = Reshape(target_shape=(relu_3.shape[1],relu_3.shape[3]))(relu_3)
    reshape_4 = Reshape(target_shape=(relu_4.shape[1],relu_4.shape[3]))(relu_4)
    reshape_5 = Reshape(target_shape=(relu_5.shape[1],relu_5.shape[3]))(relu_5)
    reshape_6 = Reshape(target_shape=(relu_6.shape[1],relu_6.shape[3]))(relu_6)

    return reshape_1,reshape_2,reshape_3,reshape_4,reshape_5,reshape_6

In [85]:




def one_stride_conv_block(reshape_1,reshape_2,reshape_3,reshape_4,reshape_5,reshape_6,filters):

    depthwiseconv1d_1 = DepthwiseConv1D(kernel_size=  4 ,padding='same')(reshape_1)
    depthwiseconv1d_2 = DepthwiseConv1D(kernel_size=  8 ,padding='same')(reshape_2)
    depthwiseconv1d_3 = DepthwiseConv1D(kernel_size= 16 ,padding='same')(reshape_3)
    depthwiseconv1d_4 = DepthwiseConv1D(kernel_size= 32 ,padding='same')(reshape_4)
    depthwiseconv1d_5 = DepthwiseConv1D(kernel_size= 64 ,padding='same')(reshape_5)
    depthwiseconv1d_6 = DepthwiseConv1D(kernel_size=128 ,padding='same')(reshape_6)

    batch_norm_1 = BatchNormalization()(depthwiseconv1d_1)
    batch_norm_2 = BatchNormalization()(depthwiseconv1d_2)
    batch_norm_3 = BatchNormalization()(depthwiseconv1d_3)
    batch_norm_4 = BatchNormalization()(depthwiseconv1d_4)
    batch_norm_5 = BatchNormalization()(depthwiseconv1d_5)
    batch_norm_6 = BatchNormalization()(depthwiseconv1d_6)

    relu_1 = ReLU()(batch_norm_1)
    relu_2 = ReLU()(batch_norm_2)
    relu_3 = ReLU()(batch_norm_3)
    relu_4 = ReLU()(batch_norm_4)
    relu_5 = ReLU()(batch_norm_5)
    relu_6 = ReLU()(batch_norm_6)

    conv1d_1 = Conv1D(filters=filters,kernel_size=  4 ,padding='same')(relu_1)
    conv1d_2 = Conv1D(filters=filters,kernel_size=  8 ,padding='same')(relu_2)
    conv1d_3 = Conv1D(filters=filters,kernel_size= 16 ,padding='same')(relu_3)
    conv1d_4 = Conv1D(filters=filters,kernel_size= 32 ,padding='same')(relu_4)
    conv1d_5 = Conv1D(filters=filters,kernel_size= 64 ,padding='same')(relu_5)
    conv1d_6 = Conv1D(filters=filters,kernel_size=128 ,padding='same')(relu_6)

    batch_norm_1 = BatchNormalization()(conv1d_1)
    batch_norm_2 = BatchNormalization()(conv1d_2)
    batch_norm_3 = BatchNormalization()(conv1d_3)
    batch_norm_4 = BatchNormalization()(conv1d_4)
    batch_norm_5 = BatchNormalization()(conv1d_5)
    batch_norm_6 = BatchNormalization()(conv1d_6)

    relu_1 = ReLU()(batch_norm_1)
    relu_2 = ReLU()(batch_norm_2)
    relu_3 = ReLU()(batch_norm_3)
    relu_4 = ReLU()(batch_norm_4)
    relu_5 = ReLU()(batch_norm_5)
    relu_6 = ReLU()(batch_norm_6)

    return relu_1,relu_2,relu_3,relu_4,relu_5,relu_6



    
def two_stride_conv_block(relu_1,relu_2,relu_3,relu_4,relu_5,relu_6,filters):

    depthwiseconv1d_1 = DepthwiseConv1D(kernel_size=  4 ,padding='same',strides= 2)(relu_1)
    depthwiseconv1d_2 = DepthwiseConv1D(kernel_size=  8 ,padding='same',strides= 2)(relu_2)
    depthwiseconv1d_3 = DepthwiseConv1D(kernel_size= 16 ,padding='same',strides= 2)(relu_3)
    depthwiseconv1d_4 = DepthwiseConv1D(kernel_size= 32 ,padding='same',strides= 2)(relu_4)
    depthwiseconv1d_5 = DepthwiseConv1D(kernel_size= 64 ,padding='same',strides= 2)(relu_5)
    depthwiseconv1d_6 = DepthwiseConv1D(kernel_size=128 ,padding='same',strides= 2)(relu_6)

    batch_norm_1 = BatchNormalization()(depthwiseconv1d_1)
    batch_norm_2 = BatchNormalization()(depthwiseconv1d_2)
    batch_norm_3 = BatchNormalization()(depthwiseconv1d_3)
    batch_norm_4 = BatchNormalization()(depthwiseconv1d_4)
    batch_norm_5 = BatchNormalization()(depthwiseconv1d_5)
    batch_norm_6 = BatchNormalization()(depthwiseconv1d_6)

    relu_1 = ReLU()(batch_norm_1)
    relu_2 = ReLU()(batch_norm_2)
    relu_3 = ReLU()(batch_norm_3)
    relu_4 = ReLU()(batch_norm_4)
    relu_5 = ReLU()(batch_norm_5)
    relu_6 = ReLU()(batch_norm_6)

    conv1d_1 = Conv1D(filters=filters,kernel_size=  4 ,padding='same')(relu_1)
    conv1d_2 = Conv1D(filters=filters,kernel_size=  8 ,padding='same')(relu_2)
    conv1d_3 = Conv1D(filters=filters,kernel_size= 16 ,padding='same')(relu_3)
    conv1d_4 = Conv1D(filters=filters,kernel_size= 32 ,padding='same')(relu_4)
    conv1d_5 = Conv1D(filters=filters,kernel_size= 64 ,padding='same')(relu_5)
    conv1d_6 = Conv1D(filters=filters,kernel_size=128 ,padding='same')(relu_6)

    batch_norm_1 = BatchNormalization()(conv1d_1)
    batch_norm_2 = BatchNormalization()(conv1d_2)
    batch_norm_3 = BatchNormalization()(conv1d_3)
    batch_norm_4 = BatchNormalization()(conv1d_4)
    batch_norm_5 = BatchNormalization()(conv1d_5)
    batch_norm_6 = BatchNormalization()(conv1d_6)

    relu_1 = ReLU()(batch_norm_1)
    relu_2 = ReLU()(batch_norm_2)
    relu_3 = ReLU()(batch_norm_3)
    relu_4 = ReLU()(batch_norm_4)
    relu_5 = ReLU()(batch_norm_5)
    relu_6 = ReLU()(batch_norm_6)

    return relu_1,relu_2,relu_3,relu_4,relu_5,relu_6


In [78]:
# input_layer = Input(shape=(256,13))

# candels = Lambda(lambda x: x[:,:,-3:])(input_layer)
# indicators = Lambda(lambda x: x[:,:,:-3])(input_layer)


# candels_reshape = Reshape(target_shape=(256,3,1))(candels)


input_layer = Input(shape=(256,3))
reshaped_candels = Reshape(target_shape=(256,3,1))(input_layer)
#Converting candels into 1d arrays

reshape_1,reshape_2,reshape_3,reshape_4,reshape_5,reshape_6 = convert_candel_to_1d_array(
                                                                                            candels_reshape=reshaped_candels,
                                                                                            filters=32
                                                                                        )



#Block1 with strides = 1, shape sizes of: 127 , 125 , 121 , 113 , 97 , 65 

relu_1,relu_2,relu_3,relu_4,relu_5,relu_6 = one_stride_conv_block(
                                                                    reshape_1,
                                                                    reshape_2,
                                                                    reshape_3,
                                                                    reshape_4,
                                                                    reshape_5,
                                                                    reshape_6,
                                                                    filters=32
                                                                )


#Block 2 with stride = 2, shape sizes of: 64 , 63 , 61 , 57 , 49 , 33
relu_1,relu_2,relu_3,relu_4,relu_5,relu_6 = two_stride_conv_block(
                                                                    relu_1,
                                                                    relu_2,
                                                                    relu_3,
                                                                    relu_4,
                                                                    relu_5,
                                                                    relu_6,
                                                                    filters=64
                                                                )

#Block 3 with stride = 2, shape sizes of: 32 , 32, 31, 29, 25 , 17
relu_1,relu_2,relu_3,relu_4,relu_5,relu_6 = two_stride_conv_block(
                                                                    relu_1,
                                                                    relu_2,
                                                                    relu_3,
                                                                    relu_4,
                                                                    relu_5,
                                                                    relu_6,
                                                                    filters=128
                                                                )

#Block 4 with stride = 2, shape sizes of: 16, 16, 16, 15, 13,  9
relu_1,relu_2,relu_3,relu_4,relu_5,relu_6 = two_stride_conv_block(
                                                                    relu_1,
                                                                    relu_2,
                                                                    relu_3,
                                                                    relu_4,
                                                                    relu_5,
                                                                    relu_6,
                                                                    filters=256
                                                                )

#Block 5 with stride = 2, shape sizes of: 8, 8, 8, 8, 7, 5
relu_1,relu_2,relu_3,relu_4,relu_5,relu_6 = two_stride_conv_block(
                                                                    relu_1,
                                                                    relu_2,
                                                                    relu_3,
                                                                    relu_4,
                                                                    relu_5,
                                                                    relu_6,
                                                                    filters=256
                                                                )


global_avg_pool_1 = keras.layers.GlobalAveragePooling1D()(relu_1)
global_avg_pool_2 = keras.layers.GlobalAveragePooling1D()(relu_2)
global_avg_pool_3 = keras.layers.GlobalAveragePooling1D()(relu_3)
global_avg_pool_4 = keras.layers.GlobalAveragePooling1D()(relu_4)
global_avg_pool_5 = keras.layers.GlobalAveragePooling1D()(relu_5)
global_avg_pool_6 = keras.layers.GlobalAveragePooling1D()(relu_6)


# print(f'{relu_1.shape=}')
# print(f'{relu_2.shape=}')
# print(f'{relu_3.shape=}')
# print(f'{relu_4.shape=}')
# print(f'{relu_5.shape=}')
# print(f'{relu_6.shape=}')

model = keras.Model(
    inputs = input_layer,

    outputs = [
                global_avg_pool_1,
                global_avg_pool_2,
                global_avg_pool_3,
                global_avg_pool_4,
                global_avg_pool_5,
                global_avg_pool_6
            ]
)



In [80]:

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_68 (InputLayer)          [(None, 256, 3)]     0           []                               
                                                                                                  
 reshape_261 (Reshape)          (None, 256, 3, 1)    0           ['input_68[0][0]']               
                                                                                                  
 conv2d_316 (Conv2D)            (None, 127, 1, 32)   416         ['reshape_261[0][0]']            
                                                                                                  
 conv2d_317 (Conv2D)            (None, 125, 1, 32)   800         ['reshape_261[0][0]']            
                                                                                            

In [10]:
# keras.utils.plot_model(model)


In [ ]:

#name of the model 
name = "4_(LSTM)_(22_03_23)"

compile_and_fit(model=model , modelname=name,
                data = data,input_window=input_window,output_window=output_window)

In [60]:
# model.save('models/secondcnn(22-03-11)/model.h5')
# model = keras.models.load_model('models/firstcnn/model.h5')

<bound method Layer.count_params of <keras.engine.functional.Functional object at 0x0000023521E5A8E0>>

In [17]:
input_test , output_test = data.make_windows(data.input_test_dataset,data.output_test_dataset)

In [18]:
predicted=model.predict(input_test)

In [21]:
predicted_list = predicted.tolist()
output_list = output_test.numpy().tolist()

In [15]:
counter = 0
for i in range(len(predicted_list)):
    if(predicted_list[i][0]>0.9 or predicted_list[i][1]>0.9):
        counter+=1

counter

6397

In [23]:
real = []
pred= []

for index in range(len(predicted_list)):
    
    if(predicted_list[index][0]>0.9):
        pred.append(predicted_list[index])
        real.append(output_list[index])
    
    elif(predicted_list[index][1]>0.9):
        pred.append(predicted_list[index])
        real.append(output_list[index])

len(real) 

6397

In [25]:
m = tf.keras.metrics.BinaryAccuracy()
m.update_state(y_true = real,y_pred = pred)
m.result().numpy()

0.5049242